<a href="https://colab.research.google.com/github/yachtsman/Coursera_Capstone/blob/master/Segmenting_and_Clustering_Neighborhoods_in_Toronto_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Segmenting and Clustering Neighborhoods in Toronto

# Mining and structuring the data

1.   Creating a new notebook
2.   Scraping data from Wikipedia
3.   Creating the dataframe
4.   Submiting the link on Github





1. Creating a new notebook

Importing basic dependencies

In [46]:
import pandas as pd
import numpy as np
print('basic dependencies imported')

basic dependencies imported


2. Scraping data from wikipedia

In [47]:
from bs4 import BeautifulSoup
import requests

print('BeautifulSoup is imported.')

BeautifulSoup is imported.


In [48]:
wiki_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(wiki_url,'lxml')
print('soup cooked')

soup cooked


In [49]:
wiki_table = soup.find('table',{'class':'wikitable sortable'})
wiki_rows = wiki_table.find_all('tr')
#wiki_rows

3. Creating the dataframe

In [50]:
data = []
for row in wiki_rows:
    data.append([t.text.strip()
    for t in row.find_all('td')]
    )
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df.drop([0], inplace=True) # to skip first rows 'none' 

print(df.shape)
df.head()

(180, 3)


,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Cleaning  the 'Not assigned' rows

In [51]:
df1 = df[df['Borough'] != 'Not assigned'].reset_index(drop=True) # to filter out 'Not assigned' rows

print(df1.shape)
df1.head()

(103, 3)


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


combinig the Neighborhoods with the equal postal codes 

In [52]:
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))
print(df2.shape)
df2.head()

(103, 2)


,Borough,Neighbourhood
PostalCode,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [53]:
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']
df3 = df2.reset_index()

In [54]:
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
print('The final cleaned table looks like:')
print(df3.shape)
df3.head()

The final cleaned table looks like:
(103, 3)


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Adding the latitude and the longitude coordinates of each neighborhood

In [55]:
csv_path = 'https://cocl.us/Geospatial_data'
df4 = pd.read_csv(csv_path)

In [56]:
print(df4.shape)
df4.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [57]:
df4.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df4.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [58]:
df_geo = pd.merge(df3, df4, how='inner')
print(df_geo.shape)
df_geo.head()

(103, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Exploring and clustering the neighborhoods in Toronto

In [59]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Observing the subject of the exploring

In [60]:
df_geo['Borough'].value_counts()

NorthYork          24
DowntownToronto    19
Scarborough        17
Etobicoke          12
CentralToronto      9
WestToronto         6
EastToronto         5
York                5
EastYork            5
Mississauga         1
Name: Borough, dtype: int64

Let's work with only boroughs that contain the word York.

In [61]:
york_data = df_geo[df_geo['Borough'].str.contains("York")].reset_index(drop=True)
print(df_geo1.shape)
york_data.head()

(34, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M2H,NorthYork,Hillcrest Village,43.803762,-79.363452
1,M2J,NorthYork,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,NorthYork,Bayview Village,43.786947,-79.385975
3,M2L,NorthYork,"York Mills, Silver Hills",43.757490,-79.374714
4,M2M,NorthYork,"Willowdale, Newtonbrook",43.789053,-79.408493


In [62]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [63]:
toronto_n = york_data
map_toronto = folium.Map(location=[toronto_n['Latitude'].mean(), 
                                toronto_n['Longitude'].mean()], 
                      zoom_start=10)
#creating a Marker for each point in df_sample. Each point will get a popup with their zip
for row in toronto_n.itertuples():
    map_toronto.add_child(folium.Marker(location=[row.Latitude ,row.Longitude],
           popup=row.Neighbourhood))

    
map_toronto

In [64]:
#@title Define Foursquare Credentials and Version

CLIENT_ID = 'ELMIMCJNZZY1KMWXSOHYO35VZWTYNLJDLHRPURI4TFGYDJTL' # your Foursquare ID
CLIENT_SECRET = '0CFJJ54D0TTSUT3KCC1D4CGHX5Y0SXMFII0DOXUQPX11PEWI' # your Foursquare Secret
VERSION = '20200706' # Foursquare API version

Let's create a function to obtain all the information about the venues from Foursquare

In [66]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [67]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [68]:
york_venues = getNearbyVenues(names=york_data['Neighbourhood'], latitudes=york_data['Latitude'], longitudes=york_data['Longitude'])

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
Bedford Park, Lawrence Manor East
Lawrence Manor, Lawrence Heights
Glencairn
Humewood-Cedarvale
Caledonia-Fairbanks
North Park, Maple Leaf Park, Upwood Park
Del Ray, Mount Dennis, Keelsdale and Silverthorn
Runnymede, The Junction North
Humber Summit
Humberlea, Emery
Weston
